### Text Generation
##### Using external dataset to get similar words
Created by Nontas

In [ ]:
#pip install -U gensim
#pip install spacy

In [17]:
# libraries
import pandas as pd
import numpy as np
import nltk
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import string
import spacy
import os 
import collections
from collections import Counter
from gensim.models.word2vec import Word2Vec
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import spacy
from nltk.tokenize.treebank import TreebankWordDetokenizer
#nltk.download()
from nltk.corpus import brown
nltk.download('word2vec_sample')
from nltk.data import find
import gensim #import models


[nltk_data] Downloading package wordnet to /Users/berry/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package word2vec_sample to
[nltk_data]     /Users/berry/nltk_data...
[nltk_data]   Package word2vec_sample is already up-to-date!


In [ ]:
data = pd.read_csv(r"../data/Cleaned_DS_Jobs.csv")
data.head()

In [ ]:
data['job_desc'] = data['Job Description'].str.lower()
data['job_desc'] = data['Job Description'].str.replace('\\n',' ')
data['job_desc'] = data['Job Description'].str.replace('http\S+|www\S+|https\S+','')
data['job_desc'] = data['Job Description'].str.replace('\S+@\S+\.\S+','')

In [ ]:
# Use this to remove unwanted in job description, kept digits 
stop_words = stopwords.words('english')
to_remove2 = stop_words + list(string.punctuation)

In [ ]:
data['job_desc_token'] = data['job_desc'].apply(word_tokenize).apply(lambda x: [item for item in x if item not in to_remove2])
data['job_desc_clean'] = data['job_desc_token'].apply(TreebankWordDetokenizer().detokenize)

### Word2vec
Build model from dataset

In [ ]:
sent_train = data.job_desc_token.values.tolist()

In [ ]:
data_model = Word2Vec(sent_train, sg = 1) #sg :Training algorithm: 1 for skip-gram; otherwise CBOW.

In [ ]:
# creating a column of bigram tokens for custom lists 
data['job_desc_bigrams'] = data['job_desc_clean'].apply(lambda row: list(nltk.ngrams(row.split(" "), 2)))
bi_list = data.job_desc_bigrams.values.tolist()

In [ ]:
data_model.wv.most_similar(['experience'])

##### Brown corpus

In [ ]:
model = gensim.models.Word2Vec(brown.sents())

In [ ]:
print(model)

In [ ]:
model.save('brown.embedding')
brown_model = gensim.models.Word2Vec.load('brown.embedding')

In [ ]:
brown_model.wv.most_similar(['experience'])

##### pre-trained NLTK model (Google News)


In [15]:
# to reset all variables in jupyter notebook
#%reset

In [18]:
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [20]:
print(model)

In [21]:
output = model.most_similar(positive=['experience'], topn = 10)
print(output)

[('experiences', 0.6655041575431824), ('expertise', 0.6086293458938599), ('knowledge', 0.5741490125656128), ('experienced', 0.529401957988739), ('familiarity', 0.5093621611595154), ('skills', 0.4901728332042694), ('Experience', 0.4896180033683777), ('perspective', 0.46464166045188904), ('background', 0.46263056993484497), ('acumen', 0.4565761685371399)]


In [26]:
df = pd.DataFrame(output, columns=['Suggestion','Percentage similar'])
print(df)

    Suggestion  Percentage similar
0  experiences            0.665504
1    expertise            0.608629
2    knowledge            0.574149
3  experienced            0.529402
4  familiarity            0.509362
5       skills            0.490173
6   Experience            0.489618
7  perspective            0.464642
8   background            0.462631
9       acumen            0.456576
